In [ ]:
import pandas as pd
from datetime import timedelta

#### Načtení tabulky 01_cruises.csv + seřazení sloupců

In [ ]:
cruises = pd.read_csv('data/01_cruises.csv')
cruises = cruises[['Dato','ETA','ETD','Navn','Passasjerer', 'Agent', 'Kai','Tonn', 'Lengde']]

#### Sjednocení názvů lodí

In [ ]:
cruises['Navn'] = cruises['Navn'].replace('Preziosa', 'MSC Preziosa')
cruises['Navn'] = cruises['Navn'].replace('Aidaluna', 'AIDAluna')
cruises['Navn'] = cruises['Navn'].replace('AidaMar', 'AIDAmar')

#### Doplnění/oprava časů u lodí, o kterých máme informace, že dopluly jindy; doplneni poctu pasazeru

In [ ]:

def corrections(df, updates):
    for row, collumn, value in updates:
        if pd.isna(df.loc[row, collumn]):
            df.loc[row, collumn] = value

# Define all updates in a list of tuples
update = [
    (9, 'Passasjerer', 2194),
    (11, 'Passasjerer', 2194),
    (16, 'ETA', '08:00'),
    (16, 'ETD', '17:00'),
    (22, 'Passasjerer', 192),
    (31, 'Passasjerer', 1338),
    (47, 'ETD', '23:00'),
    (49, 'ETA', '00:00'),
    (54, 'Passasjerer', 200),
    (66, 'Passasjerer', 192),
    (72, 'Passasjerer', 192),
    (77, 'Passasjerer', 1353),
    (101, 'Passasjerer', 192),
    (113, 'ETD', '23:00')
]

corrections(cruises, update)

In [ ]:
# # Kontrola toho, ze uz nemame v cruises hodnoty s nevyplnenym casem
lode = cruises[['Navn','Passasjerer','Dato','ETA','ETD']] #timhle si vyberu, ktere tabulky maji chybu a kuzu si to zkontrolovat v tabulce s rozpisem sluzeb, kdy lode opravdu dojely

nula = lode[lode.isna().any(axis=1)]
nula

#### Odstraneni lodi, ktere nedopluly

In [ ]:
# zbavily jsme se Queen Victoria na indexu 15, ktera nedoplula. (bude tam mozna jeste Renneisance?)
cruises = cruises.drop(15, axis='index') 

### Úprava data z formátu "dd.mm.yyyy" do formátu "yyyy-mm-dd":

In [ ]:
cruises[["Day", "Month", "Year"]] = cruises["Dato"].str.split(".", expand=True)
cruises["Date of arrival"] = cruises["Year"] + "-" + cruises["Month"]+ "-" + cruises["Day"]
cruises["Date of departure"] = cruises["Year"] + "-" + cruises["Month"]+ "-" + cruises["Day"]
cruises = cruises.drop(["Dato", "Day", "Month", "Year"], axis="columns")
cruises = cruises[["Date of arrival","Date of departure",'ETA','ETD','Navn','Passasjerer','Kai', 'Agent','Tonn', 'Lengde']]

Nové sloupečky ETA date/ETD date - slepený čas + datum příjezdu/odjezdu lodě

In [ ]:
cruises["ETA date"] = cruises["Date of arrival"] + " " + cruises["ETA"] + ":00"
cruises["ETD date"] = cruises["Date of arrival"] + " " + cruises["ETD"] + ":00"

# # Změní čas na typ datetime místo objectu, co to byl původně 
cruises["ETA date"] = pd.to_datetime(cruises["ETA date"], errors='coerce')
cruises["ETD date"] = pd.to_datetime(cruises["ETD date"], errors='coerce')

# # Vypočítá rozdíl v časech - vyhazuje to - 1 den když loď přijede před půlnocí a odjede na druhý den
cruises["Time in harbour"] = cruises["ETD date"] - cruises["ETA date"]

#index name, ať se mi na to líp hledí
cruises.index.name = "ID"


### Převedení sloupečků 'Date of arrival' a 'Date of departure' na typ datetime64[ns]

In [ ]:
cruises['Date of arrival'] = pd.to_datetime(cruises['Date of arrival'])
cruises['Date of departure'] = pd.to_datetime(cruises['Date of departure'])


### Projdi tabulku cruises a pokud je ve sloupci "Time in harbour" rozdíl méně než 0 dní 0:00:00, přičti ke sloupečku "Date of departure" jeden den: 

In [ ]:
# Definice podmínky, za které se přičte den navíc do sloupečku "Date of departure"
podminka_pridani_dne = cruises["Time in harbour"] < "0 days 00:00:00"

# Aplikace podmínky
cruises.loc[podminka_pridani_dne, "Date of departure"] = cruises["Date of departure"] + timedelta(days=1)

# zbavím se sloupečků, co nechci:
cruises = cruises.drop(["ETA date", "ETD date", "Time in harbour"], axis="columns")
cruises.head(20)
### Výsledná tabulka má date of departure o den větší než date of arrival, pokus je mezi časy příjezdu a odjezdu půlnoc

## Export do csvčka

In [ ]:
cruises.to_csv('data/cruises.csv') 
# cruises.to_csv('data/cruises.csv', index=False)  - tohle by bylo potom bez indexu